# HotpotQA预处理

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/folders/')
    !pip install transformers
    # !pip install -U spacy[cuda100]
    # !wget -P /content/folders/My\ Drive/download/ https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz
    # !pip install /content/folders/My\ Drive/download/en_core_web_lg-2.2.5.tar.gz
    # !wget -P /content/folders/My\ Drive/HotpotQA/ http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json
    # json_train_path = '/content/folders/My Drive/HotpotQA/样例_hotpot_train_v1.1.json' # 5个例子
    json_train_path = '/content/folders/My Drive/HotpotQA/hotpot_train_v1.1.json'
    save_cache_path = '/content/folders/My Drive/save_cache/'
    save_cache_path_linux = '/content/folders/My\ Drive/save_cache/'
    HotpotQA_path = '/content/folders/My Drive/HotpotQA'
except:
    json_train_path = r'./data/hotpot_train_v1.1.json'
    HotpotQA_path = './'
    save_cache_path = 'save_cache/'
    use_proxy = False
    proxies={"http_proxy": "127.0.0.1:10802",
        "https_proxy": "127.0.0.1:10802"} if use_proxy else None


In [2]:
import json
import torch
import torch.nn as nn
import numpy as np

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

with open(json_train_path, 'r', encoding='utf-8') as fp:
    json_train = json.load(fp)
    
json_train[0].keys()

dict_keys(['supporting_facts', 'level', 'question', 'context', 'answer', '_id', 'type'])

In [18]:
json_train[0]['supporting_facts']

[["Arthur's Magazine", 0], ['First for Women', 0]]

In [3]:
types = [item['type'] for item in json_train]
set(types)    

{'bridge', 'comparison'}

In [4]:
answers = [item['answer'] for item in json_train if item['type'] == 'comparison']
len(answers)

17456

In [5]:
answers[:100]

["Arthur's Magazine",
 'Jonathan Stark',
 'The Wolfhounds',
 'no',
 'no',
 'New York City',
 'Aleksander Ford',
 'yes',
 'director',
 'The Saimaa Gesture',
 'David Lee Roth',
 'no',
 'Glenn Hughes',
 'Saint Motel',
 'Boston College',
 'yes',
 'filmmaker',
 'Golf Magazine',
 'Washington, D.C.',
 'rock band',
 'yes',
 'yes',
 'MedStar Washington Hospital Center',
 'yes',
 'Turkey',
 'plant',
 'yes',
 'Mary Pierce',
 'English',
 'no',
 'Jimmie Ross',
 'rock',
 'yes',
 'Odysseas Elytis',
 'rock',
 'Richard Ford',
 'Saludos Amigos',
 'Mara Santangelo',
 'Meadowbank Gold Mine',
 'H. Bruce "Lucky" Humberstone',
 'no',
 'Mary Ramsey',
 'Jacob "Jack" Kevorkian',
 'magazine',
 'Memphis International Airport',
 'Rickenbacker International Airport',
 'playwright',
 'China',
 'plants',
 'Mark L. Lester',
 'The Raconteurs',
 'Cooking Light',
 'Virginia Wade',
 'Allan Dwan',
 'John Kennedy Toole',
 'opera',
 'yes',
 'no',
 'yes',
 'Ochagavia',
 'yes',
 'Kerkor "Kirk" Kerkorian',
 'no',
 'Chien-gris',

In [6]:
answers2 = [item['answer'] for item in json_train if item['type'] != 'comparison' and item['answer'] in ['yes', 'no']]

In [7]:
answers2

[]

## 数据统计

In [20]:
# 句子总数   
sent_len = [len((cont[1])) for ques in json_train for cont in ques['context']]
sum(sent_len)

3703344

In [21]:
# paragraph总数   
para_len = [len((ques['context'])) for ques in json_train ]
sum(para_len)

899667

In [22]:
# support sentence num:
sent_len_supp = [len(ques['supporting_facts']) for ques in json_train]
sum(sent_len_supp)

215684

In [23]:
# support para num:
para_len_supp = [len( set([i[0] for i in ques['supporting_facts']]) ) for ques in json_train]
sum(para_len_supp)

180894

In [24]:
# 问句总数
len(json_train)

90447

In [25]:
# yes_no / span
yesno_span = [1 for ques in json_train if ques['answer'] in ['yes', 'no']]
sum(yesno_span)

5481

In [3]:
# 最长的一句话 bert分词长度:652
sentenses_index_len = np.array([np.array([index1,index2,index3,len(s)]) for index1,item in enumerate(json_train) \
                 for index2,c in enumerate(item['context']) \
                 for index3,s in enumerate(c[1])])
sentenses_len = [(s,len(s)) for index1,item in enumerate(json_train) \
                 for index2,c in enumerate(item['context']) \
                 for index3,s in enumerate(c[1])]

In [4]:
max_index = np.argmax(sentenses_index_len[:,3],axis=0)

In [5]:
sentenses_index_len[max_index]

array([17519,     1,     1,  2534])

In [6]:
sentenses_len[max_index]

(" The family first appeared in 2006 and the family have been involved in a number of the show's most high-profile storylines, most notably John Paul McQueen's (James Sutton) affair with Craig Dean (Guy Burnet); Jacqui McQueen's (Claire Cooper) whirlwind relationship with Tony Hutchinson (Nick Pickard); Myra McQueen's (Nicole Barber-Lane) long-lost son Niall Rafferty's (Barry Sloane) revenge on his family by holding them hostage in an abandoned church and blowing it up, ultimately killing his half-sister Tina Reilly (Leah Hackett); Theresa McQueen's (Jorgie Porter) pregnancy by her cousin Carmel McQueen's (Gemma Merna) fiancé Calvin Valentine (Ricky Whittle) and later shooting him dead on their wedding day; Mercedes McQueen's (Jennifer Metcalfe) affair with her fiancé Riley Costello's (Rob Norbuy) father Carl (Paul Opacic); being kidnapped by Riley's grandfather Silas; staking Riley's second cousin Mitzee Minniver; Jacqui coping with the death of her husband Rhys Ashworth (Andrew Moss)

In [7]:
# 最长的整个段落 bert分词长度:1633
paragraphs_index_len = np.array([np.array([index1,index2,len(''.join(c[1]))]) for index1,item in enumerate(json_train) \
                 for index2,c in enumerate(item['context'])])

In [8]:
paragraphs_len = [(''.join(c[1]),len(''.join(c[1]))) for index1,item in enumerate(json_train) \
                 for index2,c in enumerate(item['context'])]

In [9]:
max_index_p = np.argmax(paragraphs_index_len[:,2],axis=0)
max_index_p

8369

In [10]:
paragraphs_len[max_index_p]

('Loan modification is the systematic alteration of mortgage loan agreements that help those having problems making the payments by reducing interest rates, monthly payments or principal balances. Lending institutions could make one or more of these changes to relieve financial pressure on borrowers to prevent the condition of foreclosure. Loan modifications have been practiced in the United States since The 2008 Crash Of The Housing Market from Washington Mutual, Chase Home Finance, Chase, JP Morgan & Chase, other contributors like MER\'s. Crimes of Mortgage ad Real Estate Staff had long assisted nd finally the squeaky will could not continue as their deviant practices broke the state and crashed. Modification owners either ordered by The United States Department of Housing, The United States IRS or President Obamas letters from Note Holders came to those various departments asking for the Democratic process to help them keep their homes and protection them from explosion. Thus the bi

In [11]:
max_sen_in_train = """The family first appeared in 2006 and the family have been involved in a number of the show's most high-profile storylines, most notably John Paul McQueen's (James Sutton) affair with Craig Dean (Guy Burnet); Jacqui McQueen's (Claire Cooper) whirlwind relationship with Tony Hutchinson (Nick Pickard); Myra McQueen's (Nicole Barber-Lane) long-lost son Niall Rafferty's (Barry Sloane) revenge on his family by holding them hostage in an abandoned church and blowing it up, ultimately killing his half-sister Tina Reilly (Leah Hackett); Theresa McQueen's (Jorgie Porter) pregnancy by her cousin Carmel McQueen's (Gemma Merna) fiancé Calvin Valentine (Ricky Whittle) and later shooting him dead on their wedding day; Mercedes McQueen's (Jennifer Metcalfe) affair with her fiancé Riley Costello's (Rob Norbuy) father Carl (Paul Opacic); being kidnapped by Riley's grandfather Silas; staking Riley's second cousin Mitzee Minniver; Jacqui coping with the death of her husband Rhys Ashworth (Andrew Moss) in a bus crash, learning that he had been having an affair with Cindy Cunningham (Stephanie Waring) and that he got Sinead O'Connor (Stephanie Davis) pregnant; Mercedes stalking Mitzeee (Rachel Shenton) and stabbing herself and framing her; Carmel's facial disfigurement; Myra faking her own death to escape her daughter Mercedes' evil husband, Dr. Paul Browning (Joseph Thompson); Mercedes killing her husband Doctor Browning by striking him over the head with a shovel; John Paul's male rape at the hands of his pupil Finn O'Connor (Keith Rice); the train crash which ultimately killed Carmel; Mercedes faking her death to help Grace Black (Tamara Wall) get revenge on Freddie Roscoe (Charlie Clapham); Theresa donating her kidney to Nico Blake (Persephone Swales-Dawson); Porsche (Twinnie Lee Moore) and Cleo McQueen's (Nadine Rose Mulkerrin) sexual abuse at the hands of their mother Reenie McQueen's (Zöe Lucker) fiancé Pete Buchanan (Kai Owen); Phoebe McQueen's (Mandip Gill) murder in hospital by the Gloved Hand Killer; the stillbirth of Mercedes' baby Gabriel McQueen; John Paul's transgender boss Sally St. Claire (Annie Wallace) being revealed as his biological father, Mercedes being framed for drugs by Joanne Cardsley (Rachel Leskovac), Celine McQueen (Sarah George) and Diego Salvador Martinez Hernandez De La Cruz (Juan Pablo Yepez)'s sham wedding for money and Celine being murdered by her ex-boyfriend and serial killer Cameron Campbell (Cameron Moore) after discover he causes the fire at the fair on Halloween 2016."""
max_p = 'Loan modification is the systematic alteration of mortgage loan agreements that help those having problems making the payments by reducing interest rates, monthly payments or principal balances. Lending institutions could make one or more of these changes to relieve financial pressure on borrowers to prevent the condition of foreclosure. Loan modifications have been practiced in the United States since The 2008 Crash Of The Housing Market from Washington Mutual, Chase Home Finance, Chase, JP Morgan & Chase, other contributors like MER\'s. Crimes of Mortgage ad Real Estate Staff had long assisted nd finally the squeaky will could not continue as their deviant practices broke the state and crashed. Modification owners either ordered by The United States Department of Housing, The United States IRS or President Obamas letters from Note Holders came to those various departments asking for the Democratic process to help them keep their homes and protection them from explosion. Thus the birth of Modifications. It is yet to date for clarity how theses enforcements came into existence and except b whom, but t is certain that note holders form the Midwest reached out in the Democratic Process for assistance. FBI Mortgage Fraud Department came into existence. Modifications HMAP HARP were also birthed to help note holders get Justice through reduced mortgage by making terms legal. Modification of mortgage terms was introduced by IRS staff addressing the crisis called the HAMP TEAMS that went across the United States desiring the new products to assist homeowners that were victims of predatory lending practices, unethical staff, brokers, attorneys and lenders that contributed to the crash. Modification were a fix to the crash as litigation has ensued as the lenders reorganized and renamed the lending institutions and government agencies are to closely monitor them. Prior to modifications loan holders that experiences crisis would use Loan assumptions and Loan transfers to keep the note in the 1930s. During the Great Depression, loan transfers, loan assumption, and loan bail out programs took place at the state level in an effort to reduce levels of loan foreclosures while the Federal Bureau of Investigation, Federal Trade Commission, Comptroller, the United States Government and State Government responded to lending institution violations of law in these arenas by setting public court records that are legal precedence of such illegal actions. The legal precedents and reporting agencies were created to address the violations of laws to consumers while the Modifications were created to assist the consumers that are victims of predatory lending practices. During the so-called "Great Recession" of the early 21st century, loan modification became a matter of national policy, with various actions taken to alter mortgage loan terms to prevent further economic destabilization. Due to absorbent personal profits nothing has been done to educate Homeowners or Creditors that this money from equity, escrow is truly theirs the Loan Note Holder and it is their monetary rights as the real prize and reason for the Housing Crash was the profit n obtaining the mortgage holders Escrow. The Escrow and Equity that is accursed form the Note Holders payments various staff through the United States claimed as recorded and cashed by all staff in real-estate from local residential Tax Assessing Staff, Real Estate Staff, Ordinance Staff, Police Staff, Brokers, attorneys, lending institutional staff but typically Attorneys who are also typically the owners or Rental properties that are trained through Bankruptcies\'. that collect the Escrow that is rightfully the Homeowners but because most Homeowners are unaware of what money is due them and how they can loose their escrow. Most Creditors are unaware that as the note holder that the Note Holder are due a annual or semi annual equity check and again bank or other lending and or legal intuitions staff claim this monies instead. This money Note Holders were unaware of is the prize of real estate and the cause of the Real Estate Crash of 2008 where Lending Institutions provided mortgages to people years prior they know they would eventually loose with Loan holders purchasing Balloon Mortgages lending product that is designed to make fast money off the note holder whom is always typically unaware of their escrow, equity and that are further victimized by conferences and books on HOW TO MAKE MONEY IN REAL STATE - when in fact the money is the Note Holder. The key of the crash was not the House, but the loan product used and the interest and money that was accrued form the note holders that staff too immorally. The immoral and illegal actions of predatory lending station and their staff began with the inception of balloon mortgages although illegal activity has always existed in the arena, yet the crash created "Watch Dog" like HAMP TEAM, IRS, COMPTROLLER< Federal Trade Commission Consumer Protection Bureau, FBI, CIA, Local Police Department, ICE ( The FBI online Computer crime division receives and investigates computer crimes that record keeping staff from title companies, lending institutional staff, legal staff and others created fraudulent documents to change payments and billing of note holders to obtain the money note holders are typically unaware of) and other watch dog agencies came into existence to examine if houses were purchased through a processed check at Government Debited office as many obtained free homes illegally. Many were incarcerated for such illegal actions. Modifications fixed the Notes to proper lower interest, escrow, tax fees that staff typically raised for no reason. Many people from various arenas involved in reals estate have been incarcerated for these actions as well as other illegal actions like charging for a modification. Additionally Modifications were also made to address the falsifications such as inappropriate mortgage charges, filing of fraudulently deeds, reporting of and at times filing of fraudulent mortgages that were already paid off that were fraudulently continued by lenders staff and attorneys or brokers or anyone in the Real Estate Chain through the issues of real estate terms to continue to violate United States Laws, contract law and legal precedence where collusion was often done again to defraud and steal from the Note Holder was such a common practice that was evidence as to why the Mortgage Crash in 2008 occurred for the purpose of wining the prize of stealing form Homeowners and those that foreclosed was actually often purposefully for these monies note holders were unaware of to be obtained which was why Balloon mortgages and loans were given to the staff in the Real Estate Market with the hoper and the expectation that the loan holders would default as it offered opportunity to commit illegal transactions of obtaining the homeowners funds. While such scams were addressed through modifications in 2008. The Market relied heavily on Consumers ignorance to prosper, ignorance of real estate terms, ignorance on what they were to be charged properly for unethical financial gain and while staff in real estates lending arenas mingled terms to deceive y deliberate confusion consumers out of cash and homes while the USA Government provided Justice through President Obamas Inception and IRS Inception of Modifications which addressed these unethical profits in Reals Estate. It was in 2009 that HARP, HAMP and Modifications were introduced to stop the victimization of Note Holders. Taking on the Banks that ran USA Government was a great and dangerous undertaking that made America Great Again as Justice for Consumers reigned. Legal action taken against institutions that have such business practices can be viewed in State Code of Law and Federal Law on precedent cases that are available to the public. Finally, It had been unlawful to be charged by an attorney to modify as well as fro banking staff to modify terms to increase a mortgage and or change lending product to a balloon in an concerted effort to make homeowner foreclose which is also illegal, computer fraud and not the governments intended purpose or definition of a modification.'


## 使用BERT进行分词与NER

应该先分词, 再获取NER span.

**注意!** 使用GPU加速则不能使用多进程.

**注意!** 不同模型的分词格式有差异. 不是所有模型都支持NER.

In [6]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

model_NER = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english",
                                                           proxies = proxies)
tokenizer = AutoTokenizer.from_pretrained("bert-large-cased-whole-word-masking",
                                         proxies = proxies)

_ = model_NER.to(DEVICE)

In [7]:
def tokensizer_in_Model(content_raw, special=False, tokenizer = tokenizer):
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(content_raw)))
    if special: return tokens
    else: return tokens[1:-1]

In [8]:
s = "Echosmith is an American, Corporate indie pop band formed in February 2009 in Chino, California."
print(tokensizer_in_Model(s, special=True, tokenizer=tokenizer))
# print(tokensizer_in_Model(s, special=True, tokenizer=tokenizer2))

['[CLS]', 'Echo', '##smith', 'is', 'an', 'American', ',', 'Corporate', 'indie', 'pop', 'band', 'formed', 'in', 'February', '2009', 'in', 'Chin', '##o', ',', 'California', '.', '[SEP]']


In [9]:
def find_NER_in_Model(content_raw, tokens=None, model=model_NER, tokenizer = tokenizer):
    '''返回: tokens[1:-1], inputs, entities_list
    第一个是不包含[CLS] [SEP]的分词序列(content_tokens是包含的).
    第二个是实体列表.'''
    label_list = [
        "O",       # Outside of a named entity
        "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
        "I-MISC",  # Miscellaneous entity
        "B-PER",   # Beginning of a person's name right after another person's name
        "I-PER",   # Person's name
        "B-ORG",   # Beginning of an organisation right after another organisation
        "I-ORG",   # Organisation
        "B-LOC",   # Beginning of a location right after another location
        "I-LOC"    # Location
        ]
    # Bit of a hack to get the tokens with the special tokens
    if not tokens: tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(content_raw)))
    inputs = tokenizer.encode(content_raw, return_tensors='pt').to(DEVICE)
    outputs = model_NER(inputs)[0]
    predictions = torch.argmax(outputs, dim=2)
    # 去除 [cls] [sep]
    res = [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())][1:-1]
    # print(predictions)
    # print(res)

    entities_list = []
    cursor_1 = 0
    cursor_2 = 0

    while cursor_1 < len(res):
        entities_dict = {}
        temp = []
        if res[cursor_1][1] == 'O': 
            cursor_1+=1
            continue
        
        entities_dict['type'] = res[cursor_1][1]
        entities_dict['span_start'] = cursor_1
        temp.append(res[cursor_1][0])
        cursor_2 = cursor_1 + 1
        while cursor_2 < len(res):
            if res[cursor_2][1] == 'O':
                cursor_1 = cursor_2 + 1
                break
            
            temp.append(res[cursor_2][0])
            cursor_2 += 1
        
        cursor_1 += cursor_2
        entities_dict['content'] = ' '.join(temp).replace(' ##', '')
        entities_dict['content_tokens'] = temp
        entities_list.append(entities_dict)

    return tokens[1:-1], entities_list

In [10]:
print(find_NER_in_Model(s, model=model_NER, tokenizer=tokenizer))
# print(find_NER_in_Model(s, model=model_NER_2, tokenizer=tokenizer2))

(['Echo', '##smith', 'is', 'an', 'American', ',', 'Corporate', 'indie', 'pop', 'band', 'formed', 'in', 'February', '2009', 'in', 'Chin', '##o', ',', 'California', '.'], [{'type': 'I-ORG', 'span_start': 0, 'content': 'Echosmith', 'content_tokens': ['Echo', '##smith']}, {'type': 'I-LOC', 'span_start': 15, 'content': 'Chino', 'content_tokens': ['Chin', '##o']}])


## 使用spacy进行分词与NER


## class Node


In [20]:
class BaseNode(object):
    '''Node class for graph'''
    
    def __init__(self, node_id, node_type, parent_id, \
            content_raw, content_tokens, content_features=None, cls_feature=None):
        
        self.node_id = node_id
        self.node_type = node_type
        self.parent_id = parent_id

        self.content_raw = content_raw
        self.content_tokens = content_tokens

        self.content_features = content_features
        self.cls_feature = None # final features. [1,dim]
        
    @classmethod
    def build(cls):
        raise NotImplementedError

    def __str__(self):
        raise NotImplementedError
    
    def __repr__(self):
        raise NotImplementedError


class QuestionNode(BaseNode):
    def __init__(self,node_id, node_type, parent_id, content_raw, content_tokens,\
                    answer, ques_type):
        super(QuestionNode, self).__init__(node_id, node_type, parent_id, content_raw, content_tokens)

        self.question = self.content_raw
        self.answer = answer
        self.ques_type = ques_type

    @classmethod
    def build(cls, node_id, parent_id, content_raw, content_tokens,
                answer, ques_type):

        node_type = 'Question'
        return cls(node_id, node_type, parent_id, content_raw, content_tokens,\
                    answer, ques_type)
    
    def __str__(self):
        return f'QuestionNode: {self.node_id}'
    
    def __repr__(self):
        return self.__str__()


class ParagraphTitleNode(BaseNode):
    '''不含整个段落, 只含有title'''
    def __init__(self,node_id, node_type, parent_id, content_raw, content_tokens,\
                    content_NER_list = None):
        super(ParagraphTitleNode, self).__init__(node_id, node_type, parent_id, content_raw, content_tokens)

        self.content_NER_list = content_NER_list

        self.paragraph_id = -99
        self.start_in_paragraph = -99
        self.end_in_paragraph = -99

        self.is_support = None # 段落 句子 

    @classmethod
    def build(cls, node_id, parent_id, content_raw, content_tokens):

        content_tokens, content_NER_list = find_NER_in_Model(content_raw, content_tokens)
        # content_tokens: NO CLS.

        node_type = 'Paragraph'
        return cls(node_id, node_type, parent_id, content_raw, content_tokens,\
                    content_NER_list)
    
    def set_support(self):
        self.is_support = True

    def set_span_in_paragraph(self, para_id, start):
        self.paragraph_id = para_id
        self.start_in_paragraph = start
        self.end_in_paragraph = start + len(self.content_tokens)

    def __str__(self):
        return f'ParagraphTitleNode: {self.node_id}'
    
    def __repr__(self):
        return self.__str__()


class SentenceNode(BaseNode):
    def __init__(self,node_id, node_type, parent_id, content_raw, content_tokens,\
                    content_NER_list = None):
        super(SentenceNode, self).__init__(node_id, node_type, parent_id, content_raw, content_tokens)

        self.content_NER_list = content_NER_list

        self.paragraph_id = -99
        self.start_in_paragraph = -99
        self.end_in_paragraph = -99

        self.is_support = None # 段落 句子 

    @classmethod
    def build(cls, node_id, parent_id, content_raw, content_tokens=None):

        content_tokens, content_NER_list = find_NER_in_Model(content_raw, content_tokens)
        # content_tokens: NO CLS.

        node_type = 'Sentence'
        return cls(node_id, node_type, parent_id, content_raw, content_tokens,\
                    content_NER_list)

    def set_span_in_paragraph(self, para_id, start):
        self.paragraph_id = para_id
        self.start_in_paragraph = start
        self.end_in_paragraph = start + len(self.content_tokens)

    def set_support(self):
        self.is_support = True

    def get_NER_tuples_list(self):
        '''返回NER元组. e.g. [('ALLPE',id), ('DELL',id)]'''
        return [(i['content'], self.node_id) for i in self.content_NER_list]

    def __str__(self):
        return f'SentenceNode: {self.node_id}'
    
    def __repr__(self):
        return self.__str__()


class EntityNode(BaseNode):
    def __init__(self,node_id, node_type, parent_id, content_raw, content_tokens,\
                    ):
        super(EntityNode, self).__init__(node_id, node_type, parent_id, content_raw, content_tokens)

        self.start_in_sentence = -99
        self.end_in_sentence = -99

        self.paragraph_id = -99
        self.start_in_paragraph = -99
        self.end_in_paragraph = -99

    @classmethod
    def build(cls, node_id, parent_id, content_raw, content_tokens=None):

        node_type = 'Entity'
        return cls(node_id, node_type, parent_id, content_raw, content_tokens, \
                    )
    
    # only for E_node.
    def set_span_in_sentence(self, start):
        self.start_in_sentence = start
        self.end_in_sentence = start + len(self.content_tokens)

    def set_span_in_paragraph(self, para_id, start):
        self.paragraph_id = para_id
        self.start_in_paragraph = start
        self.end_in_paragraph = start + len(self.content_tokens)

    def __str__(self):
        return f'EntityNode: {self.node_id}'
    
    def __repr__(self):
        return self.__str__()



## class Question_Paragraph

辅助函数`get_feature_from_model`应放入py文件.

In [12]:
class Question_Paragraph(object):
    '''Q-P pair and label. for BERT and node init.
    返回q-p对和q-s对, 还要确保能够初始化Node类.
    每个问句有10个paragraph,即10个此类.'''
    def __init__(self, ques_id, para_id, question_tokens, para_title_tokens, para_label, sents_in_para, sentences_label):
        self.question_tokens = question_tokens
        self.para_title_tokens = para_title_tokens
        self.sents_in_para = sents_in_para
        self.sentences_offsets = self.cal_offsets(self.sents_in_para)
        self.sentences_label = sentences_label

        self.ques_id = ques_id
        self.para_id = para_id 
        self.para_label = para_label # 段落label

        self.question_features = None # [N, dim]
        self.para_features = None
#         self.paragraph_features = None

    @classmethod
    def build(cls, ques_id, para_id, question, para_title_tokens, para_label, node_list):
        Snodes = [n for n in node_list if n.paragraph_id == para_id and n.node_type == 'Sentence']
        question_tokens = tokensizer_in_Model(question)
        # para_title_tokens = tokensizer_in_Model(para_title)
        sents_in_para = [n.content_tokens for n in Snodes]
        sentences_label = [int(n.is_support) for n in Snodes]
        return cls(ques_id, para_id, question_tokens, para_title_tokens, para_label, sents_in_para, sentences_label)

    @staticmethod
    def cal_offsets(sents_list):
        cursor = 0
        offsets = []
        for sent_tokens in sents_list:
            len_sent = len(sent_tokens)
            offsets.append((cursor, cursor+len_sent))
            cursor += len_sent
        return offsets
    
    # content tokens
    def get_para_tokens(self, contain_title = False):
        para_token = self.para_title_tokens if contain_title else []
        for i in self.sents_in_para: para_token.extend(i)
        return para_token

    def get_ques_para_label_tuple(self, contain_title = False):
        '''问句-段落对'''
        return (self.question_tokens, self.get_para_tokens(contain_title), self.para_label)

    def get_ques_sent_label_list(self, contain_title = False):
        '''问句-句子对'''
        if contain_title:
            return [(self.question_tokens, self.para_title_tokens.extend(sent_tokens), sent_label)\
                for sent_tokens,sent_label in zip(self.sents_in_para, self.sentences_label)]
        else:
            return [(self.question_tokens, sent_tokens, sent_label)\
                for sent_tokens,sent_label in zip(self.sents_in_para, self.sentences_label)]

    def format_sents_in_para(self):
        return ' '.join([f'{index}:{word}' for index,word in enumerate(self.sents_in_para)])

    # features
    def build_features(self):
        '''build features from LM models'''
    
        self.question_features = get_feature_from_model(self.question_tokens)

        para_features = 0
        for one_line in self.get_ques_sent_label_list():

            sent_features = get_feature_from_model(one_line[0], one_line[1], 'second')

            if type(para_features) == int: para_features = sent_features.clone()
            else: para_features = torch.cat((para_features, sent_features), dim=0)
       
        self.para_features = para_features
        

    def get_question_features(self):
        return self.question_features

    def get_paragraph_features(self):
        return self.para_features

    # other
    def __str__(self):
        return f'Q_P. p_id: {self.para_id}'
    
    def __repr__(self):
        return f'Q_P. p_id: {self.para_id}'


## class Adjacency_sp

In [13]:
import scipy.sparse as sp
import numpy as np

class Adjacency_sp(object):
    '''无重复稀疏邻接矩阵'''
    def __init__(self):
        self.v_i_j = []
        self.i_j_find_table = []

    def append(self, v, i, j):
        if not (i,j) in self.i_j_find_table:
          self.v_i_j.append([v,i,j])
          self.i_j_find_table.append((i,j))
    
    def to_dense(self, shape=(200, 200)):
        '''return numpy ndarray.'''
        np_adj = np.array(self.v_i_j)
        full_adj = sp.coo_matrix((np_adj[:, 0], (np_adj[:, 1], np_adj[:, 2])), shape=shape, dtype=np.float32).todense()
        full_adj = np.array(full_adj)
        return full_adj

    def to_dense_symmetric(self, shape=(200, 200)):
        np_adj = np.array(self.v_i_j)
        adj = sp.coo_matrix((np_adj[:, 0], (np_adj[:, 1], np_adj[:, 2])), shape=shape, dtype=np.float32)
        adj_symm = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj).todense()
        adj_symm = np.array(adj_symm)
        return adj_symm

    def __repr__(self):
        return f'Adjacency_sp has {len(self.v_i_j)} edges'
    def __str__(self):
        return f'Adjacency_sp has {len(self.v_i_j)} edges'
    def __len__(self):
        return len(self.v_i_j)

## 开始

**4种点**

1. Question
2. Paragraph
3. Sentence
4. Entity

In [14]:
# 返回Q-paragraph(for BERT); adj; node_list
edge_type_map = {
    'Q_P':1,
    'Q_E':2,
    'P_S':3,
    'S_S_hyper':4,
    'S_E':5,
    'P_P':6,
    'S_S':7,
}

def create_ques_info_dict():
    # 每个question item由5个元素组成
    ques_info_dict={}
    ques_info_dict['id'] = None
    ques_info_dict['node_list'] = None
    ques_info_dict['sp_adj'] = None
    ques_info_dict['ques_para_list'] = None
    return ques_info_dict

### sub_func

In [21]:
from collections import defaultdict
from traceback import print_exc

def _process(item):
    '''sub-function for multi-processes function.'''
    ques_info_dict = create_ques_info_dict()

    supporting_facts = defaultdict(list)
    for s_fact in item['supporting_facts']:
        supporting_facts[s_fact[0]].append(s_fact[1])

    level = item['level']
    question = item['question']
    context = item['context']
    answer = item['answer']
    Q_id = item['_id']
    Q_type = item['type']

    ques_info_dict['id'] = Q_id

    node_list = []
    sp_adj = Adjacency_sp()
    index_cursor = 0
    
    Q_node_cursor = index_cursor    
    ques_tokens = tokensizer_in_Model(question, special=False)
    
    # node_id, parent_id, content_raw, content_tokens
    Q_node = QuestionNode.build(Q_node_cursor, -1, question, ques_tokens,\
                               answer = answer, ques_type = Q_type)
    
    node_list.append(Q_node)

    ques_para_list = []
    
    for p_index, paragraph in enumerate(context):
        # 通过`para_uu_id`在另一个列表(Q-P类列表)中查找对应的元素.
        para_uu_id = f'{Q_id}_{p_index}'
        offset_in_paragraph = 0
        paragraph_label = 0

        title = paragraph[0]
        title_tokens = tokensizer_in_Model(title, special=True)
        
        index_cursor += 1

        # 添加P-1和P
        try:
            if index_cursor != 0: sp_adj.append(edge_type_map['P_P'], P_node_cursor, index_cursor) 
        except (NameError): # 首次调用P_node_cursor会报错
            pass
        P_node_cursor = index_cursor
        P_node = ParagraphTitleNode.build(P_node_cursor, Q_node_cursor, title, title_tokens)
        P_node.set_span_in_paragraph(para_uu_id, offset_in_paragraph)
        
        # 判断 support paragraph
        if title in supporting_facts.keys():
            paragraph_label = 1
            P_node.set_support()
        node_list.append(P_node)
        
        # 添加Q和P
        sp_adj.append(edge_type_map['Q_P'], Q_node_cursor, P_node_cursor)

        
        for s_index, sentence in enumerate(paragraph[1]):
            S_id = f'{Q_id}_{p_index}_{s_index}'
            sentence_tokens = tokensizer_in_Model(sentence, special=True)

            index_cursor += 1
            S_node_cursor = index_cursor
            S_node = SentenceNode.build(S_node_cursor, P_node_cursor, sentence, sentence_tokens)
            S_node.set_span_in_paragraph(para_uu_id, offset_in_paragraph)
            
            # 判断support fact
            if (paragraph_label == 1) and (s_index in supporting_facts[title]):
                S_node.set_support()
            node_list.append(S_node)

            # 添加S之间边; P和S之间边
            if s_index != 0:
                sp_adj.append(edge_type_map['S_S'], S_node_cursor - 1, S_node_cursor)
            sp_adj.append(edge_type_map['P_S'], P_node_cursor, S_node_cursor)

            # 添加S和E之间边
            for entities_dict in S_node.content_NER_list:
                index_cursor += 1
                E_node_cursor = index_cursor
                E_node = EntityNode.build(E_node_cursor, S_node_cursor, \
                                          entities_dict['content'], entities_dict['content_tokens'])
                E_node.set_span_in_paragraph(para_uu_id,offset_in_paragraph+entities_dict['span_start'])
                E_node.set_span_in_sentence(entities_dict['span_start'])

                node_list.append(E_node)
                sp_adj.append(edge_type_map['S_E'], S_node_cursor, E_node_cursor)
            
            offset_in_paragraph += len(sentence_tokens[1:-1])

        # in paragraph loop.
        ques_para = Question_Paragraph.build(Q_id, para_uu_id, question, title_tokens[1:-1], paragraph_label, node_list)
        ques_para_list.append(ques_para)
    
    # in item loop
    # 连接Q节点和E节点.
    E_nodes_in_Q = [i for i in node_list if i.node_type == 'Entity' and i.content_raw.replace(' ','') \
                    in question.replace(' ','')]
    for i in E_nodes_in_Q:
        sp_adj.append(edge_type_map['Q_E'], Q_node_cursor, i.node_id)
    
    # 连接S和P节点
    # 注意!!原文没有使用entity linking系统, 而是直接使用Wikipedia提供的链接.
    # 这里暂时使用字符串match.
    S_node = [i for i in node_list if i.node_type == 'Sentence']
    E_nodes = [i for i in node_list if i.node_type == 'Entity']
    for E_n in E_nodes:
        entity = E_n.content_raw.replace(' ','')
        for S_n in S_node:
            if entity in S_n.content_raw.replace(' ',''):
                sp_adj.append(edge_type_map['S_S_hyper'], node_list[E_n.parent_id].node_id, node_list[S_n.parent_id].node_id)
            
    
    ques_info_dict['node_list'] = node_list
    ques_info_dict['sp_adj'] = sp_adj
    ques_info_dict['ques_para_list'] = ques_para_list
    return ques_info_dict

### main func

In [22]:
from tqdm import tqdm_notebook
def preprocessing(item_num = 5):
    '''main multi-processes function.'''
    item_num = None if item_num<0 else item_num
    
    resturn_list = [] 
    for item in tqdm_notebook(json_train[:item_num]):
        resturn_list.append(_process(item))
    
    return resturn_list

In [23]:
hotpotQA_train_preprocess = preprocessing(100)

AttributeError: 'QuestionNode' object has no attribute 'paragraph_id'

## 检查

### E,S和Q-P类之间的对应关系.

In [31]:
def check_span_offset_in_QP(index_of_ques, index_of_X, Node_Type):
    assert Node_Type in ['Entity', 'Sentence', 'Paragraph']
    assert hotpotQA_train_preprocess

    node_list = hotpotQA_train_preprocess[index_of_ques]['node_list']
    ques_para_list = hotpotQA_train_preprocess[index_of_ques]['ques_para_list']

    X_node = [i for i in node_list if i.node_type == Node_Type]
    print(f"len {Node_Type} node: {len(X_node)}")
    test_start = int(X_node[index_of_X].start_in_paragraph)
    test_end = int(X_node[index_of_X].end_in_paragraph)
    print(X_node[index_of_X].paragraph_id)
    print(X_node[index_of_X].content_raw)
    r = [i for i in ques_para_list if i.para_id == X_node[index_of_X].paragraph_id][0].get_para_tokens()
    print(r)
    print(f'offset in paragraph: \n{r[test_start:test_end]}')

In [32]:
check_span_offset_in_QP(2, 6, 'Entity')

len Entity node: 115
5a8d7341554299441c6b9fe5_0
James L . Brooks
['Lisa', 'Marie', 'Simpson', 'is', 'a', 'fictional', 'character', 'in', 'the', 'animated', 'television', 'series', '"', 'The', 'Simpsons', '"', '.', 'She', 'is', 'the', 'middle', 'child', 'and', 'most', 'intelligent', 'of', 'the', 'Simpson', 'family', '.', 'Voiced', 'by', 'Year', '##dley', 'Smith', ',', 'Lisa', 'first', 'appeared', 'on', 'television', 'in', '"', 'The', 'Trace', '##y', 'U', '##ll', '##man', 'Show', '"', 'short', '"', 'Good', 'Night', '"', 'on', 'April', '19', ',', '1987', '.', 'Cartoon', '##ist', 'Matt', 'G', '##ro', '##ening', 'created', 'and', 'designed', 'her', 'while', 'waiting', 'to', 'meet', 'James', 'L', '.', 'Brooks', '.', 'G', '##ro', '##ening', 'had', 'been', 'invited', 'to', 'pitch', 'a', 'series', 'of', 'shorts', 'based', 'on', 'his', 'comic', '"', 'Life', 'in', 'Hell', '"', ',', 'but', 'instead', 'decided', 'to', 'create', 'a', 'new', 'set', 'of', 'characters', '.', 'He', 'named', 'the', 'elde

In [33]:
check_span_offset_in_QP(2, 2, 'Sentence')

len Sentence node: 68
5a8d7341554299441c6b9fe5_0
 Voiced by Yeardley Smith, Lisa first appeared on television in "The Tracey Ullman Show" short "Good Night" on April 19, 1987.
['Lisa', 'Marie', 'Simpson', 'is', 'a', 'fictional', 'character', 'in', 'the', 'animated', 'television', 'series', '"', 'The', 'Simpsons', '"', '.', 'She', 'is', 'the', 'middle', 'child', 'and', 'most', 'intelligent', 'of', 'the', 'Simpson', 'family', '.', 'Voiced', 'by', 'Year', '##dley', 'Smith', ',', 'Lisa', 'first', 'appeared', 'on', 'television', 'in', '"', 'The', 'Trace', '##y', 'U', '##ll', '##man', 'Show', '"', 'short', '"', 'Good', 'Night', '"', 'on', 'April', '19', ',', '1987', '.', 'Cartoon', '##ist', 'Matt', 'G', '##ro', '##ening', 'created', 'and', 'designed', 'her', 'while', 'waiting', 'to', 'meet', 'James', 'L', '.', 'Brooks', '.', 'G', '##ro', '##ening', 'had', 'been', 'invited', 'to', 'pitch', 'a', 'series', 'of', 'shorts', 'based', 'on', 'his', 'comic', '"', 'Life', 'in', 'Hell', '"', ',', 'but'

### E和S之间的对应关系.

In [34]:
import numpy as np
def check_span_offset_in_S(index_of_ques, choose_num, seed=1234):
    Node_Type = 'Entity'
    assert hotpotQA_train_preprocess

    node_list = hotpotQA_train_preprocess[index_of_ques]['node_list']

    X_node = [i for i in node_list if i.node_type == Node_Type]
    print(f"len {Node_Type} node: {len(X_node)}")

    np.random.seed(seed)
    choose_nodes = np.random.choice(X_node, choose_num)

    for N_index,N in enumerate(choose_nodes):
        print(f'N: {N_index}')
        print(f'E_node:\n\t{N.content_raw}')
        test_start = int(N.start_in_sentence)
        test_end = int(N.end_in_sentence)

        N_parent = node_list[N.parent_id]
        print(f'in S_node:\n\t{N_parent.content_tokens[test_start:test_end]}')
        print(f'{N_parent.content_tokens}\n\n')

In [35]:
check_span_offset_in_S(2, 3)

len Entity node: 115
N: 0
E_node:
	Everything ' s Coming Up
in S_node:
	['Everything', "'", 's', 'Coming', 'Up']
['She', 'is', 'co', '-', 'host', 'of', 'Everything', "'", 's', 'Coming', 'Up', 'Po', '##d', '##cast', ',', 'a', 'Simpsons', '-', 'focused', 'podcast', 'along', 'with', 'Julia', 'Prescott', '.']


N: 1
E_node:
	The Tracey Ullman Show
in S_node:
	['The', 'Trace', '##y', 'U', '##ll', '##man', 'Show']
['After', 'appearing', 'for', 'three', 'seasons', 'on', '"', 'The', 'Trace', '##y', 'U', '##ll', '##man', 'Show', '"', ',', 'the', 'Simpson', 'family', 'got', 'their', 'own', 'series', 'on', 'Fox', 'that', 'debuted', 'December', '17', ',', '1989', '.']


N: 2
E_node:
	Goertz
in S_node:
	['Go', '##ert', '##z']
['Go', '##ert', '##z', 'is', 'known', 'for', 'her', 'satirical', 'songs', 'based', 'on', 'various', 'pop', 'culture', 'topics', '.']




## 保存为pkl

In [36]:
import pickle

In [37]:
with open(save_cache_path+'hotpotQA_train_preprocess100.pkl', 'wb') as fp:
    pickle.dump(hotpotQA_train_preprocess, fp, protocol=-1)
!ls -hl $save_cache_path_linux

total 4.0M
-rw-r--r-- 1 root root 8.3K Mar  8 17:41 Classes.py
drwxr-xr-x 2 root root  246 Mar  7 00:53 data
-rw-r--r-- 1 root root  71K Mar  6 20:12 HotpotQA_preprocessing.ipynb
-rw-r--r-- 1 root root 3.6M Mar  6 17:51 hotpotQA_train_process100.pkl
-rw-r--r-- 1 root root 6.4K Mar  6 17:51 main.py
drwxr-xr-x 2 root root   36 Mar  8 16:57 __pycache__
-rw-r--r-- 1 root root  956 Mar  6 17:51 README.en.md
-rw-r--r-- 1 root root 1.3K Mar  6 17:51 README.md
drwxr-xr-x 2 root root   72 Mar  8 11:33 save_cache
-rw-r--r-- 1 root root  181 Mar  6 17:51 start_jupyter_lab_here.bat
-rw-r--r-- 1 root root 143K Mar  9 10:15 step1_HotpotQA预处理封装.ipynb
-rw-r--r-- 1 root root  55K Mar  8 11:39 step2_train_on_BERT.ipynb
-rw-r--r-- 1 root root  33K Mar  9 10:13 step3_Build_node_features.ipynb
-rw-r--r-- 1 root root 6.5K Mar  7 13:17 untitled.txt
-rw-r--r-- 1 root root  46K Mar  9 09:19 语言模型测试.ipynb
